## Importing Libraries


In [ ]:
# Importing Libraries
import os
import random
import math
import sys
import gc


import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold


import torch
import albumentations
from albumentations.pytorch import ToTensorV2
import torch.nn.functional as F
from torch.nn import BCEWithLogitsLoss
from torch.utils.data import DataLoader
from torch.optim import Adam


sys.path.append("../input/timmmaster/")
from timm import create_model

In [ ]:
# Constants  
img_size = 384
Batch_size = 8
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epochs = 2

# Directories
train_dir = "/kaggle/input/petfinder-pawpularity-score/train/"
test_dir = "/kaggle/input/petfinder-pawpularity-score/test/"


def seed_everything():
    os.environ['PYTHONHASHSEED'] = str(123)
    np.random.seed(123)
    random.seed(123)
    os.environ["TF_CPP_MIN_LOG_LEVEL"] = '2'
    os.environ['PYTHONHASHSEED'] = str(123)

seed_everything()

## Loading and Processing Data

In [ ]:
# Reading dataset train, test in df and df_test respectively
df = pd.read_csv("/kaggle/input/petfinder-pawpularity-score/train.csv")
df_test = pd.read_csv("/kaggle/input/petfinder-pawpularity-score/test.csv")
Id = df_test["Id"].copy()


# Converting Id column for taking images
df["Id"] = df["Id"].apply(lambda x : "/kaggle/input/petfinder-pawpularity-score/train/" + x + ".jpg")
df_test["Id"] = df_test["Id"].apply(lambda x : "/kaggle/input/petfinder-pawpularity-score/test/" + x + ".jpg")


# Dropping unwanted columns and shuffling dataset
dense_features = ["Subject Focus", "Eyes", "Face", "Near", "Action", "Accessory", 
                 "Group", "Collage", "Human", "Occlusion", "Info", "Blur"]
df = df.sample(frac = 1).reset_index(drop = True)

# For treating problem as classification
df["Pawpularity"] = df["Pawpularity"] / 100.0

In [ ]:
df.sample(5)

## Preparing Dataset

In [ ]:
# Defining Dataset class
class Dataset:
    def __init__(self, img_paths, targets, augmentations, mixup = False):
        self.img_paths = img_paths
        self.targets = targets
        self.augmentations = augmentations
        self.mixup = mixup
    
    def __len__(self):
        return len(self.img_paths)
    
    def __getitem__(self, item):
        if self.mixup: # For Mixup
            image1 = cv2.imread(self.img_paths[item])
            image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
            target1 = self.targets[item]

            img_idx = random.randint(0, len(self.img_paths)-1)
            image2 = cv2.imread(self.img_paths[img_idx])
            image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)
            target2 = self.targets[img_idx]
            
            if self.augmentations is not None:
                augmented = self.augmentations(image=image1)
                image1 = augmented["image"]
                augmented = self.augmentations(image=image2)
                image2 = augmented["image"]

            alpha = 0.8
            lam = np.random.beta(alpha, alpha)
            image = lam * image1 + (1 - lam) * image2
            target = lam * target1 + (1- lam) * target2
            
        else: # Without Mixup
            image = cv2.imread(self.img_paths[item])
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            if self.augmentations is not None:
                augmented = self.augmentations(image=image)
                image = augmented["image"]
                
            target = self.targets[item]
        
        return {
            "images" : image,
            "labels" : torch.tensor(target, dtype=torch.float)
        }

In [ ]:
# For augmentation
aug = albumentations.Compose(
    [
        albumentations.Resize(img_size, img_size, p=1),
        albumentations.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
            max_pixel_value=255.0,
            p=1.0,
        ),
        albumentations.HueSaturationValue(
            hue_shift_limit=0.2, sat_shift_limit=0.2,
            val_shift_limit=0.2, p=0.5
        ),
        albumentations.RandomBrightnessContrast(
            brightness_limit=(-0.1, 0.1),
            contrast_limit=(-0.1, 0.1), p=0.5
        ),
        ToTensorV2(p=1.0)
    ]
)

test_aug = albumentations.Compose(
    [
        albumentations.Resize(img_size, img_size, p=1),
        albumentations.Normalize(
              mean=[0.485, 0.456, 0.406],
              std=[0.229, 0.224, 0.225],
              max_pixel_value=255.0,
          ),
        ToTensorV2(p=1.0)
    ]
)

In [ ]:
# Creating bins using Struges rule
num_bins = int(1 + (3.3 * np.log2(len(df))))

df["bins"] = pd.cut(df["Pawpularity"], bins = num_bins, labels = False)
df["bins"].hist()

In [ ]:
# Creating folds
df['fold'] = -1


N_FOLDS = 5
strat_kfold = StratifiedKFold(n_splits=N_FOLDS, random_state=1024, shuffle=True)
for i, (_, train_index) in enumerate(strat_kfold.split(df.index, df['bins'])):
    df.iloc[train_index, -1] = i
    
df['fold'] = df['fold'].astype('int')

In [ ]:
# Creating Test datset
test_dataset = Dataset(
    img_paths = df_test["Id"].values,
    targets = np.ones(len(df_test)),
    augmentations = test_aug
)

test_loader = DataLoader(test_dataset, batch_size = 1) # batch_size = 1 for predicting values one by one

## Creating Model

In [ ]:
# Defining model
def swin():
    return create_model("swin_large_patch4_window12_384", pretrained = True, num_classes = 1)

In [ ]:
# Defining RMSE score and sigmoid functions
def petfinder_rmse(inp,target):
    return 100*torch.sqrt(F.mse_loss(torch.sigmoid(inp.flatten()), target))

def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [ ]:
# Defining model class 
class Pawpularity_model:
    def __init__(self, criterion, optimizer, metrics, scheduler):
        self.criterion = criterion
        self.optimizer = optimizer
        self.metrics = metrics
        self.scheduler = scheduler
    
    def train_batch_loop(self, model, train_loader):
        train_loss = 0 
        train_acc = 0

        for data in tqdm(train_loader):
            images = data["images"].to(device)
            labels = data["labels"].to(device)

            logits = model(images)
            loss = criterion(logits.flatten(), labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()
            
            train_loss += loss.item()
            train_acc += metrics(logits, labels).item()
        
        return train_loss / len(train_loader), train_acc / len(train_loader)
    
    def valid_batch_loop(self, model, valid_loader):
        valid_loss = 0 
        valid_acc = 0

        with torch.no_grad():
            for data in tqdm(valid_loader):
                images = data["images"].to(device)
                labels = data["labels"].to(device)

                logits = model(images)
                loss = criterion(logits.flatten(), labels)

                valid_loss += loss.item()
                valid_acc += metrics(logits, labels).item()
        
        return valid_loss / len(valid_loader), valid_acc / len(valid_loader)
    
    def fit(self, model, train_loader, valid_loader, epochs):

        for i in range(epochs):

            model.train()
            avg_train_loss, avg_train_acc = self.train_batch_loop(model, train_loader)
            

            model.eval()
            avg_valid_loss, avg_valid_acc = self.valid_batch_loop(model, valid_loader)

            print("Epoch : {} ".format(i+1))
            print("Train Loss : {:.6f} Train MSE : {:.6f}".format(avg_train_loss, avg_train_acc))
            print("Valid Loss : {:.6f} Valid MSE : {:.6f}".format(avg_valid_loss, avg_valid_acc),"\n")

## Training and Evaluating Model

In [ ]:
# Training model
final_pred = []
for i in range(N_FOLDS):
    print("#" * 20)
    print("#### FOLD {}".format(i+1))
    print("#" * 20, "\n")
    train = df[df["fold"] != i].reset_index(drop = True)
    valid = df[df["fold"] == i].reset_index(drop = True)
    
    # Creating Train and Validation dataset
    train_dataset = Dataset(
        img_paths = train["Id"].values,
        targets = train["Pawpularity"].values,
        augmentations = aug,
        mixup = True
    )

    valid_dataset = Dataset(
        img_paths = valid["Id"].values,
        targets = valid["Pawpularity"].values,
        augmentations = test_aug,
    )

    train_loader = DataLoader(train_dataset, batch_size = Batch_size, shuffle = True,drop_last = True)
    valid_loader = DataLoader(valid_dataset, batch_size = Batch_size, shuffle = True,drop_last = True)
    
    
    model = swin().to(device)
    criterion = BCEWithLogitsLoss()
    optimizer = Adam(model.parameters(), lr = 1e-4)
    metrics = petfinder_rmse
    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0 = 10, T_mult=1, eta_min=1e-6)

    
    trainer = Pawpularity_model(criterion, optimizer, metrics, scheduler)
    trainer.fit(model, train_loader, valid_loader, epochs)
    
    torch.save(model, f"swin_model_f{i}.bin")
    
    # Predicting for Test dataset
    print("=> Predicting Test Dataset","\n")
    pred = []
    for data in test_loader:
        images = data["images"].to(device)
        pred.append(sigmoid(model(images).item()) * 100) 
    
    final_pred.append(pred)
    
    del model
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
# Mean of predicted values
np.mean(np.stack(final_pred))

In [ ]:
# Creating submission file
ans = np.mean(np.stack(final_pred), axis=0)

final = pd.DataFrame()
final["Id"] = Id
final["Pawpularity"] = ans
final.to_csv("submission.csv", index=False)

In [ ]:
final.head()